In [1]:
import sys

sys.path.append("..")
from omegaconf import OmegaConf
from sgm.util import instantiate_from_config

config = OmegaConf.load(
    "/vol/paramonos2/projects/antoni/code/Personal/generative-models/configs/example_training/svd_training.yaml"
)
print(config)
config["model"]["params"][
    "ckpt_path"
] = "/vol/paramonos2/projects/antoni/code/Personal/generative-models/checkpoints/svd.safetensors"
config["model"]["params"]["network_config"]["params"]["fine_tuning_method"] = None
model = instantiate_from_config(config.model)

{'model': {'base_learning_rate': 3e-05, 'target': 'sgm.models.diffusion.DiffusionEngine', 'params': {'input_key': 'latents', 'no_log_keys': ['audio_emb', 'fps_id', 'motion_bucket_id', 'cond_aug'], 'scale_factor': 0.18215, 'disable_first_stage_autocast': True, 'ckpt_path': 'checkpoints/svd.safetensors', 'denoiser_config': {'target': 'sgm.modules.diffusionmodules.denoiser.Denoiser', 'params': {'scaling_config': {'target': 'sgm.modules.diffusionmodules.denoiser_scaling.VScalingWithEDMcNoise'}}}, 'network_config': {'target': 'sgm.modules.diffusionmodules.video_model.VideoUNet', 'params': {'adm_in_channels': 768, 'num_classes': 'sequential', 'use_checkpoint': True, 'in_channels': 8, 'out_channels': 4, 'model_channels': 320, 'attention_resolutions': [4, 2, 1], 'num_res_blocks': 2, 'channel_mult': [1, 2, 4, 4], 'num_head_channels': 64, 'use_linear_in_transformer': True, 'transformer_depth': 1, 'context_dim': 1024, 'spatial_transformer_attn_type': 'softmax-xformers', 'extra_ff_mix_layer': True

In [ ]:
from safetensors.torch import load_file

model_ckpt = load_file("../checkpoints/svd.safetensors")

print(model_ckpt.keys())

In [2]:
unet = model.model.diffusion_model

In [ ]:
from sgm.modules.diffusionmodules.adapters.lora import apply_lora
import torch

# activate_lora = add_lora_to(unet, ["SpatialVideoTransformer"], search_class=[torch.nn.Linear], r=32, dropout=0, lora_bias="none")
apply_lora(unet, filters=[".input_blocks"], rank=16, all_modules_in_filter=True)

In [ ]:
from sgm.modules.diffusionmodules.adapters.lora import LoRA

lora = {}
lora["weights"] = torch.nn.ModuleList()
for module in unet.modules():
    if isinstance(module, LoRA) or (
        hasattr(module, "_fsdp_wrapped_module") and isinstance(module._fsdp_wrapped_module, LoRA)
    ):
        print(module.requires_grad_())
        lora["weights"].append(module)

In [ ]:
lora

In [ ]:
for param in unet.parameters():
    if param.requires_grad:
        print("requires grad")
    else:
        print("does not require grad")

In [ ]:
for p in unet.input_blocks[0].parameters():
    print(p)

In [9]:
for name, module in unet.named_modules():
    print(name)


time_embed
time_embed.0
time_embed.1
time_embed.2
label_emb
label_emb.0
label_emb.0.0
label_emb.0.1
label_emb.0.2
input_blocks
input_blocks.0
input_blocks.0.0
input_blocks.1
input_blocks.1.0
input_blocks.1.0.in_layers
input_blocks.1.0.in_layers.0
input_blocks.1.0.in_layers.1
input_blocks.1.0.in_layers.2
input_blocks.1.0.h_upd
input_blocks.1.0.emb_layers
input_blocks.1.0.emb_layers.0
input_blocks.1.0.emb_layers.1
input_blocks.1.0.out_layers
input_blocks.1.0.out_layers.0
input_blocks.1.0.out_layers.1
input_blocks.1.0.out_layers.2
input_blocks.1.0.out_layers.3
input_blocks.1.0.skip_connection
input_blocks.1.0.time_stack
input_blocks.1.0.time_stack.in_layers
input_blocks.1.0.time_stack.in_layers.0
input_blocks.1.0.time_stack.in_layers.1
input_blocks.1.0.time_stack.in_layers.2
input_blocks.1.0.time_stack.h_upd
input_blocks.1.0.time_stack.emb_layers
input_blocks.1.0.time_stack.emb_layers.0
input_blocks.1.0.time_stack.emb_layers.1
input_blocks.1.0.time_stack.out_layers
input_blocks.1.0.time_

In [16]:
from peft import LoraModel, LoraConfig

lora_config = LoraConfig(
    inference_mode=False,
    r=8,
    lora_alpha=32,
    lora_dropout=0.1,
    target_modules=["output_blocks.10.1.time_stack.0.attn2.to_q"],
)
lora_unet = LoraModel(unet, lora_config, "bite")

In [17]:
loraunet

LoraModel(
  (model): VideoUNet(
    (time_embed): Sequential(
      (0): Linear(in_features=320, out_features=1280, bias=True)
      (1): SiLU()
      (2): Linear(in_features=1280, out_features=1280, bias=True)
    )
    (label_emb): Sequential(
      (0): Sequential(
        (0): Linear(in_features=768, out_features=1280, bias=True)
        (1): SiLU()
        (2): Linear(in_features=1280, out_features=1280, bias=True)
      )
    )
    (input_blocks): ModuleList(
      (0): TimestepEmbedSequential(
        (0): Conv2d(8, 320, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
      )
      (1-2): 2 x TimestepEmbedSequential(
        (0): VideoResBlock(
          (in_layers): Sequential(
            (0): GroupNorm32(32, 320, eps=1e-05, affine=True)
            (1): SiLU()
            (2): Conv2d(320, 320, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
          )
          (h_upd): Identity()
          (x_upd): Identity()
          (emb_layers): Sequential(
            (0): SiLU(